In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(np.__version__)

In [ ]:
import pandas as pd
import re
import numpy as np
from pandas.core.frame import DataFrame
from tensorflow.python.framework.indexed_slices import _LARGE_SPARSE_NUM_ELEMENTS
from tensorflow.python.keras.backend import sigmoid
import tensorflow as tf

from numpy.core.defchararray import title
from pandas.core.arrays import categorical
import sklearn.preprocessing
DEFAULT_FILE_PATH = "/kaggle/input/filepython/glove.6B.50d.txt"

In [ ]:

def df_process(df=None,train=False,model='rnn_glove'): # train arguement can be used somewhere
    if df is None:
        df = pd.read_csv("google-quest-challenge/train.csv",index_col='qa_id')
    if(model=='rnn_glove'):
        #Extract id of user (maybe same person always get same kind of score) {for overlaying logic}
        df['question_user_page']=df['question_user_page'].apply(lambda x: x.split("/")[-1])
        df['answer_user_page']=df['answer_user_page'].apply(lambda x: x.split("/")[-1])
        #print(df['category'].unique())
        df['category']=df['category'].apply(lambda x:re.sub("[^a-zA-Z]"," ",x).lower())
        #df['category']=df['category'].str.replace('life arts',)
        df['answer_user_page']=df['answer_user_page'].apply(lambda x: x.split("/")[-1])
        #extract site name on which question posted
        df['host']=df['host'].apply(lambda x: x.split(".")[0])
        df.host=pd.Categorical(df.host)
        df['host'] = df.host.cat.codes
        df.category=pd.Categorical(df.category)
        df['category'] = df.category.cat.codes
        #print(df['host'].unique()) #remove host column, not needed
        #dropping useless columns
        df=df.drop(['question_user_name','answer_user_name','url'],axis=1)
        #create new features: question title length, question length, answer length
        df['q_len']=df['question_body'].str.len()
        df['t_len']=df['question_title'].str.len()
        df['ans_len']=df['answer'].str.len()
       
        return df




In [ ]:
def loadWordVectors(tokens, filepath=DEFAULT_FILE_PATH, dimensions=50): #tokens is the set of words in training data
    """Read pretrained GloVe vectors"""
    wordVectors = np.random.randn(len(tokens), dimensions) #random normal distribution so words not in glove get random embeddings
    with open(filepath) as ifs:
        for line in ifs:
            line = line.strip()
            if not line:
                continue
            row = line.split()
            token = row[0]
            if token not in tokens:
                continue
            data = [float(x) for x in row[1:]]
            if len(data) != dimensions:
                raise RuntimeError("wrong number of dimensions")
            wordVectors[tokens[token]] = np.asarray(data)
    return wordVectors

In [ ]:
class wordbank:
    def __init__(self,df=None,train=True):
        if df is None:
            df = pd.read_csv("google-quest-challenge/train.csv",index_col='qa_id')
        self.df=df
        self.train=train

    def tokens(self):
        if hasattr(self, "_tokens") and self._tokens: # if already attribute present 
            print("Token attribute present already")
            return self._tokens

        tokens = dict()
        tokenfreq = dict()
        wordcount = 0
        idx = 0
        tokens["<pad>"] = idx
        tokenfreq["<pad>"] = 0 
        wordcount += 1
        idx += 1
        self._sentences=self.questions()+self.answers()+self.titles()
        # create common token dict
        for sentence in self._sentences:
            for w in sentence:
                wordcount += 1
                if not w in tokens:
                    tokens[w] = idx
                    tokenfreq[w] = 1
                    idx += 1
                else:
                    tokenfreq[w] += 1

        

        self._tokens = tokens
        self._tokenfreq = tokenfreq
        self._wordcount = wordcount
        self._titles=[[tokens[word] for word in sent] for sent in self.titles()]
        self._questions=[[tokens[word] for word in sent] for sent in self.questions()]
        self._answers=[[tokens[word] for word in sent] for sent in self.answers()]
        return self._tokens

    '''Different functions if diffferent type of preprocessing required'''

    def answers(self):
        if hasattr(self, "_answers") and self._answers:
            return self._answers

        ans_df=self.df['answer']
        self._use_answers = ans_df.tolist()
        ans_df=ans_df.apply(lambda x: (re.sub("[^a-zA-Z]"," ",x)).strip().split())
        ans_df=ans_df.apply(lambda x: [w.lower() for w in x])
        self._answers = ans_df.tolist()
        return self._answers
    def titles(self):
        if hasattr(self, "_titles") and self._titles:
            return self._titles
        
        titles_df=self.df['question_title']
        self._use_titles = titles_df.tolist()
        titles_df=titles_df.apply(lambda x: (re.sub("[^a-zA-Z]"," ",x)).strip().split())
        titles_df=titles_df.apply(lambda x: [w.lower() for w in x])
        self._titles = titles_df.tolist()
        return self._titles
    def questions(self):
        if hasattr(self, "_questions") and self._questions:
            return self._questions

        q_df=self.df['question_body']
        self._use_questions = q_df.tolist()
        q_df=q_df.apply(lambda x: (re.sub("[^a-zA-Z]"," ",x)).strip().split())
        q_df=q_df.apply(lambda x: [w.lower() for w in x])
        self._questions = q_df.tolist()
        return self._questions

    def numSentences(self):
        if hasattr(self, "_numSentences") and self._numSentences:
            return self._numSentences
        else:
            self._numSentences = len(self._answers())
            return self._numSentences
    def numQ(self): # numbe of unique questions or titles
        if hasattr(self, "_numQ") and self._numQ:
            return self._numQ
        else:
            self._numQ = self.df['question_title'].nunique()
            return self._numQ
    def test(self): # for test time
        ind_unk=self._tokens['unknown']
        self._titles=[[self._tokens.get(word,ind_unk) for word in sent] for sent in self.titles()]
        self._questions=[[self._tokens.get(word,ind_unk) for word in sent] for sent in self.questions()]
        self._answers=[[self._tokens.get(word,ind_unk) for word in sent] for sent in self.answers()]
    
        

In [ ]:
# Source: https://stackoverflow.com/questions/53404301/how-to-compute-spearman-correlation-in-tensorflow
from scipy.stats import spearmanr
def get_spearman_rankcor(y_true, y_pred,flag=False):
    if not flag:
        
        a=np.zeros(30)
        for i in range(30):
            a[i]=np.array((tf.py_function(spearmanr, [tf.cast(y_pred[:,i], tf.float32), 
                       tf.cast(y_true[:,i], tf.float32)], Tout = tf.float32)))
        return a.mean()
        
            
    return ( tf.py_function(spearmanr, [tf.cast(y_pred, tf.float32), 
                       tf.cast(y_true, tf.float32)], Tout = tf.float32) )

In [ ]:
'''First Model'''
class NNGoogleQuest:
    def __init__(self,data,vocab,embedding_matrix=None,max_length=[0,0,0],method='lstm'):
        self._data=data
        self._vocab=vocab
        self._embedding_matrix=embedding_matrix
        self._embedding_dim=self._embedding_matrix.shape[1]
        self._num_vocab=len(self._vocab)
        self._max_length=max_length
        self._method=method
    def embeddings(self,input,ind):
        embedding_layer = tf.keras.layers.Embedding(
            self._num_vocab,
            self._embedding_dim,
            embeddings_initializer=tf.keras.initializers.Constant(self._embedding_matrix),
            trainable=True,mask_zero=True,input_length=self._max_length[ind])(input)
        
        #print(embedding_layer.shape)
        if(self._method=='lstm'):
            lstm_layer=tf.keras.layers.SimpleRNN(64)(embedding_layer)
            #self._embedding_layer=embedding_layer
            return lstm_layer
        else:
            cov_layer=tf.keras.layers.Conv1D(64, 3, activation='relu')(embedding_layer)
            #print(cov_layer.shape)
            
            flat=tf.keras.layers.GlobalMaxPooling1D()(cov_layer)
            #flat=tf.keras.layers.Flatten()(cov_layer)
            return flat
    def NN_model(self):
        
        title_input = tf.keras.Input(shape=(None,), name="title")  # Variable-length sequence of ints
        question_input = tf.keras.Input(shape=(None,), name="question")  # Variable-length sequence of ints
        answer_input=tf.keras.Input(shape=(None,), name="answer") # Variable-length sequence of ints
        category_input = tf.keras.Input(shape=(5,), name="category")  # category binary input vector (5 categories)
        #host_input=tf.keras.Input(shape=(59,), name="host") #host category(59 total)
        stats_input=tf.keras.Input(shape=(3,),name='stats') #new created features
        if (self._method=='lstm'):
            title_state_h = self.embeddings(title_input,0)

            question_state_h = self.embeddings(question_input,1)
            answer_state_h = self.embeddings(answer_input,2)
        else:
            title_state_h = self.embeddings(title_input,0)

            question_state_h= self.embeddings(question_input,1)
            answer_state_h= self.embeddings(answer_input,2)
        
        features= tf.keras.layers.concatenate([title_state_h, question_state_h, answer_state_h,category_input,stats_input])
        hidden=tf.keras.layers.Dense(128,activation='relu',name='hidden')(features)
        drop1=tf.keras.layers.Dropout(0.4)(hidden)
        pred=tf.keras.layers.Dense(30,activation='sigmoid',name='prediction')(drop1)
        model = tf.keras.Model(inputs=[title_input, question_input, answer_input,category_input,stats_input],outputs=pred)
        self._model=model
        if(self._method=='lstm'):
            tf.keras.utils.plot_model(self._model, "rnn_simple__model.png", show_shapes=True)
        else:
            tf.keras.utils.plot_model(self._model, "cnn_model.png", show_shapes=True)
        
    
    '''def answerEmbeddings(self):  
    def questionEmbeddings(self):
    def titleEmbeddings(self):
    def categoryEmbeddings(self):'''
    
    def train(self):
        self.NN_model()
        self._model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='Adam'
        ),
        loss='mean_absolute_error')
        self._model.fit(
        {"title": self._data['title'], "question": self._data['question'], "answer": self._data['answer'],"category":self._data['category'],"stats":self._data['stats']},
        {"prediction": self._data['output']},
        epochs=20,
        batch_size=128,
            validation_split=0.1
        )
        if(self._method=='lstm'):
            self._model.save('rnn_glove_model')
        else:
            self._model.save('cnn_glove_model')
        #self._model=tf.keras.models.load_model('rnn_glove_model')
        #df = pd.DataFrame(np.array(self._model.predict({"title": self._data['title'], "question": self._data['question'], "answer": self._data['answer'],"category":self._data['category'],"host":self._data['host'],"stats":self._data['stats']},batch_size=32)))
        #df.to_csv('submission.csv') 

**Train using LSTM and CNN models**

In [ ]:
def multi_hot_enc(array,label=None):
    label_binarizer = sklearn.preprocessing.LabelBinarizer()
    label_binarizer.fit(range(max(train[label].to_numpy())+1)) # for test
    array=label_binarizer.transform(array)
    return array
def normalize(df):
    x = df.values #returns a numpy array
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return x_scaled

train=pd.read_csv("/kaggle/input/google-quest-challenge/train.csv",index_col='qa_id') #6079 rows
train=df_process(train,True)
Gq=wordbank(train)
tokens=Gq.tokens()

word_vectors=loadWordVectors(tokens)
titles_vec=Gq._titles
#title_vec = tf.keras.preprocessing.sequence.pad_sequences(titles_vec, padding="post")
questions_vec=Gq._questions
answers_vec=Gq._answers
'''if not any(Gq._titles+Gq._answers+Gq._questions):
    print("Empty list!")
#word_vectors=tf.convert_to_tensor(word_vectors)'''
data={} # train data as dictionary
data['question']=tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences(
    questions_vec, padding="post"))
data['answer']=tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences(
    answers_vec, padding="post"))
data['title']=tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences(
    titles_vec, padding="post"))
#data['host']=tf.convert_to_tensor(multi_hot_enc(train['host'].to_numpy(),'host'))
data['category']=tf.convert_to_tensor(multi_hot_enc(train['category'].to_numpy(),'category'))
data['stats']=tf.convert_to_tensor(normalize(train[['q_len','t_len','ans_len']]))
data['output']=tf.convert_to_tensor(train.iloc[:,7:37].values)


In [ ]:
print('Training using GRU model\n')
model=NNGoogleQuest(data,tokens,word_vectors,method='lstm',max_length=[data['title'].shape[1],data['question'].shape[1],data['answer'].shape[1]])
model.train()


In [ ]:
print('Training using simple RNN model\n')
model=NNGoogleQuest(data,tokens,word_vectors,method='lstm',max_length=[data['title'].shape[1],data['question'].shape[1],data['answer'].shape[1]])
model.train()

**Loading Models**

In [ ]:
'''rnn_model=tf.keras.models.load_model('../input/notebookd751344e59/rnn_glove_model',custom_objects={'get_spearman_rankcor':get_spearman_rankcor})'''


In [ ]:
'''cnn_model=tf.keras.models.load_model('../input/notebookd751344e59/cnn_glove_model',custom_objects={'get_spearman_rankcor':get_spearman_rankcor})'''

**Test dataset**

In [ ]:
test=pd.read_csv("/kaggle/input/google-quest-challenge/test.csv",index_col='qa_id')
test2=test.copy()
test2=df_process(test2,False)
GqTest=wordbank(test2,train=False)
GqTest._tokens=tokens # train tokens

GqTest.test()
titles_vecT=GqTest._titles
#title_vec = tf.keras.preprocessing.sequence.pad_sequences(titles_vec, padding="post")
questions_vecT=GqTest._questions
answers_vecT=GqTest._answers
dataT={}
dataT['question']=tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences(
    questions_vecT, padding="post"))
dataT['answer']=tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences(
    answers_vecT, padding="post"))
dataT['title']=tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences(
    titles_vecT, padding="post"))
#dataT['host']=tf.convert_to_tensor(multi_hot_enc(test2['host'].to_numpy(),'host'))
dataT['category']=tf.convert_to_tensor(multi_hot_enc(test2['category'].to_numpy(),'category'))
dataT['stats']=tf.convert_to_tensor(normalize(test2[['q_len','t_len','ans_len']]))
'''df = pd.DataFrame(np.array(cnn_model.predict(
    {"title": dataT['title'], "question": dataT['question'], "answer": dataT['answer'],"category":dataT['category'],"stats":dataT['stats']})))
df.index=test.index
test[list(train.iloc[:,7:37].columns)]=df.iloc[:,0:30]
test.to_csv('submission.csv') '''

In [ ]:
'''test=test.drop(['question_title', 'question_body','question_user_name','question_user_page','answer','answer_user_name','answer_user_page','url','category','host'], axis = 1)
test.to_csv('submission.csv') '''

**Summary of RNN and LSTM models**

In [ ]:
'''rnn_model.summary()'''

In [ ]:
'''cnn_model.summary()'''

In [ ]:
#rnn_model.get_weights()

In [ ]:
#rnn_model.get_layer('lstm_18').get_weights()
#cnn_model.input

**Extracting features from trained RNN-LSTM model and training using other models**

In [ ]:
'''#extracting middle outputs from saved model
rnn_embed=tf.keras.Model(inputs=rnn_model.input,outputs=rnn_model.get_layer('concatenate_6').output)
test_features=rnn_embed.predict({"title": dataT['title'], "question": dataT['question'], "answer": dataT['answer'],"category":dataT['category'],"stats":dataT['stats']})
train_features=rnn_embed.predict({"title": data['title'], "question": data['question'], "answer": data['answer'],"category":data['category'],"stats":data['stats']})
print('Shape of middle features:',train_features.shape)'''

In [ ]:
'''from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
train_y=data['output']
train_X, val_X, train_y, val_y = train_test_split(np.array(train_features), np.array(train_y),
                      test_size = 0.1, random_state = 123)
arr=np.zeros(30)
for i in range(30):
    
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(train_X,train_y[:,i])
    pred=regressor.predict(val_X)
    print('Spearman score for predicting y',i,'=',get_spearman_rankcor(val_y[:,i],pred,True))
    arr[i]=np.array(get_spearman_rankcor(val_y[:,i],pred,True))
print('Mean of Spearman score: ',arr.mean() )'''

In [ ]:
'''import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE'''


**Extracting features from trained CNN model and training using other models**

In [ ]:
'''#extracting middle outputs from saved model
cnn_embed=tf.keras.Model(inputs=cnn_model.input,outputs=cnn_model.get_layer('concatenate_5').output)
test_features=cnn_embed.predict({"title": dataT['title'], "question": dataT['question'], "answer": dataT['answer'],"category":dataT['category'],"stats":dataT['stats']})
train_features=cnn_embed.predict({"title": data['title'], "question": data['question'], "answer": data['answer'],"category":data['category'],"stats":data['stats']})
print('Shape of middle features:',train_features.shape)'''

In [ ]:
'''train_y=data['output']
train_X, val_X, train_y, val_y = train_test_split(np.array(train_features), np.array(train_y),
                      test_size = 0.1, random_state = 123)
arr=np.zeros(30)
for i in range(30):
    
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(train_X,train_y[:,i])
    pred=regressor.predict(val_X)
    print('Spearman score for predicting y',i,'=',get_spearman_rankcor(val_y[:,i],pred,True))
    arr[i]=np.array(get_spearman_rankcor(val_y[:,i],pred,True))
print('Mean of Spearman score: ',arr.mean() )'''

In [ ]:
# ref: https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder
import tensorflow_hub as tfhub
import seaborn as sns

**Universal Sentence Encoder**

In [ ]:

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_model = tfhub.load(module_url)
def use_embed(input):
    return use_model(input)

In [ ]:
#preprocessing handled by USE automatically, mentioned in the docs
use_titles=use_embed(Gq._use_titles) # 512 dimesnional vector
use_questions=use_embed(Gq._use_questions)
use_answers=use_embed(Gq._use_answers)

In [ ]:
use_titles=tf.expand_dims(use_titles, axis=1)
use_questions=tf.expand_dims(use_questions, axis=1)
use_answers=tf.expand_dims(use_answers, axis=1)
print(use_questions.shape)

In [ ]:
title_input = tf.keras.Input(shape=(1,512), name="title")  # Variable-length sequence of ints
question_input = tf.keras.Input(shape=(1,512), name="question")  # Variable-length sequence of ints
answer_input=tf.keras.Input(shape=(1,512), name="answer") # Variable-length sequence of ints
category_input = tf.keras.Input(shape=(5,), name="category")  # category binary input vector (5 categories)
#host_input=tf.keras.Input(shape=(59,), name="host") #host category(59 total)
stats_input=tf.keras.Input(shape=(3,),name='stats') #new created features
concat=tf.keras.layers.concatenate([title_input, question_input,answer_input],axis=-2)
cov_layer=tf.keras.layers.Conv1D(48, 3, activation='relu')(concat)
flat=tf.keras.layers.Flatten()(cov_layer)
concat2=tf.keras.layers.concatenate([flat,category_input,stats_input],axis=-1)
drop=tf.keras.layers.Dropout(0.4)(concat2)
hidden=tf.keras.layers.Dense(512,activation='relu',name='hidden')(drop)
drop2=tf.keras.layers.Dropout(0.4)(hidden)
pred=tf.keras.layers.Dense(30,activation='sigmoid',name='prediction')(drop2)
'''model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(512,),activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(30,activation='sigmoid'))'''

model = tf.keras.Model(inputs=[title_input, question_input, answer_input,category_input,stats_input],outputs=pred)
tf.keras.utils.plot_model(model, "use_model_logistic.png", show_shapes=True)
model.summary()

In [ ]:

model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='Adam'
        ),
        loss='mean_absolute_error')
model.fit(
        {"title": use_titles, "question": use_questions, "answer": use_answers,"category":data['category'],"stats":data['stats']},
        {"prediction": data['output']},
        epochs=100,
        batch_size=48,
            validation_split=0.1
        )

In [ ]:
pred=model.predict({"title": use_titles, "question": use_questions, "answer": use_answers,"category":data['category'],"stats":data['stats']})

In [ ]:
get_spearman_rankcor(data['output'], pred)

**BERT**